In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ast import literal_eval
from collections import Counter
import itertools

pd.set_option('display.max_colwidth', 180)
pd.set_option('expand_frame_repr', False)

In [13]:
csvfile = '../proc_data_phase0/verification/all_responses_round0-3_verified.csv'
df = pd.read_csv(csvfile, 
    sep="\t")

total_objects=len(df)
print("Total objects: ", total_objects)
df.columns

Total objects:  25315


Index(['vg_img_id', 'cat', 'synset', 'vg_obj_name', 'responses_r0', 'opt-outs',
       'vg_domain', 'top_response_domain_r0', 'responses_domains_r0', 'url',
       'sample_type', 'responses_domains_r1', 'responses_r1',
       'top_response_domain_r1', 'responses_domains_r2', 'responses_r2',
       'top_response_domain_r2', 'responses_domains_r3', 'responses_r3',
       'top_response_domain_r3', 'vg_object_id', 'all_responses', 'clean',
       'canon', 'spellchecked', 'spellchecked_min2', 'verified'],
      dtype='object')

In [50]:
#df['clean'] = df['clean'].apply(lambda x: eval(x))
#df['verified'] = df['verified'].apply(lambda x: eval(x))
df['canon'] = df['canon'].apply(lambda x: eval(x))


In [49]:
df['canon'][0]

"Counter({'man': 11, 'batter': 8, 'player': 5, 'person': 4, 'baseball player': 2, 'helmet': 2, 'boy': 1, 'uniform': 1, 'bat': 1})"

In [80]:
df['mn_name'] = df['canon'].apply(lambda x: x.most_common(1)[0][0])

In [83]:
vocab = set(df.mn_name)

In [84]:
vocab

{'airplane',
 'airport',
 'ambulance',
 'animal',
 'apple',
 'apron',
 'aquarium',
 'arm',
 'athlete',
 'awning',
 'baby',
 'backpack',
 'bacon',
 'bagel',
 'balloon',
 'banana',
 'bar',
 'barn',
 'baseball player',
 'basketball player',
 'bat',
 'bathtub',
 'batter',
 'beach',
 'bear',
 'beard',
 'bed',
 'beer',
 'bench',
 'benches',
 'bidet',
 'bike',
 'bird',
 'bird bath',
 'birdbath',
 'biscuit',
 'blanket',
 'blender',
 'board',
 'boat',
 'book',
 'bookshelf',
 'booth',
 'bottle',
 'bowl',
 'bowtie',
 'box',
 'boy',
 'bread',
 'bricks',
 'bride',
 'bridge',
 'broccoli',
 'bucket',
 'buffet',
 'bug',
 'building',
 'bull',
 'bun',
 'bunk bed',
 'bunkbed',
 'burger',
 'burrito',
 'bus',
 'bus stop',
 'butterfly',
 'cabbage',
 'cabinet',
 'cabinets',
 'cake',
 'camera',
 'can',
 'canister',
 'canopy',
 'cape',
 'car',
 'carpet',
 'carriage',
 'carrot',
 'cart',
 'castle',
 'cat',
 'catcher',
 'cereal',
 'chair',
 'cheese',
 'cheetah',
 'chicken',
 'child',
 'chili',
 'china',
 'church

In [85]:
len(vocab)

445

In [51]:

error_cats = Counter()

for ix,row in df.iterrows():
    resp = row['canon']
    topname = resp.most_common(1)[0][0]
    #print(topname)
    for name in resp:
        if name == topname:
            error_cats['hit'] += resp[name]
        elif name not in row['verified']:
            error_cats['low_count'] += resp[name]
        else:
            if topname in row['verified'][name]['can_be_same_object']:
                error_cats['adequate_same_object'] += resp[name]
            else:
                if not row['verified'][name]['inadequacy_type']:
                    error_cats['adequate_diff_object'] += resp[name]
                else:
                     error_cats[row['verified'][name]['inadequacy_type']] += resp[name]

                    
                    
                    #print('img:',row['vg_img_id'],'--entry-level:',topname)
                    #print('less-preferred:',name)
                    #print(row['verified'][name])
            
    #if ix > 5:       
    #    break
        

In [52]:
error_cats

Counter({'hit': 616370,
         'adequate_same_object': 143690,
         'bounding box': 42971,
         'low_count': 80846,
         'other': 696,
         'adequate_diff_object': 7949,
         'visual': 1040,
         'linguistic': 106})

In [53]:
for t in error_cats:
    print(t,error_cats[t]/sum(error_cats.values()))

hit 0.6897080347511604
adequate_same_object 0.160786779877986
bounding box 0.04808385216881437
low_count 0.09046536297595975
other 0.0007788127134461568
adequate_diff_object 0.008894802096528017
visual 0.0011637431350344871
linguistic 0.00011861228107082273


In [103]:
csvfile = '../papers/ACL2020/material_experiments/FRCNN--VG1600--VGMN_MN.tsv'
csvfile2 = '../papers/ACL2020/material_experiments/FRCNN--VG1600_VG.tsv'

dfsys = pd.read_csv(csvfile, 
    sep="\t")

total_objects=len(dfsys)
print("Total objects: ", total_objects)
dfsys.columns

dfsys2 = pd.read_csv(csvfile2, 
    sep="\t")

total_objects=len(dfsys2)
print("Total objects: ", total_objects)
dfsys2.columns

Total objects:  1071
Total objects:  1071


Index(['image_id', 'category', 'mn_name', 'adequacy', 'pred_name',
       'adequacy.1', 'pred_score_converted'],
      dtype='object')

In [104]:
dfsys

,image_id,category,mn_name,adequacy,pred_name,adequacy.1,pred_score_converted
0,32,hit,cabinet,1.000000,cabinet,1.0,0.005396
1,160,hit,dress,1.000000,dress,1.0,0.005525
2,308,hit,bridge,1.000000,bridge,1.0,0.005896
3,474,error,table,1.000000,bed,--,0.005595
4,535,hit,table,1.000000,table,1.0,0.007436
...,...,...,...,...,...,...,...
1066,2417438,hit,bed,0.900000,bed,0.9,0.006303
1067,2417585,error,airplane,1.000000,bench,--,0.003789
1068,2417682,hit,curtain,1.000000,curtain,1.0,0.014026
1069,2417758,hit,man,0.833333,man,0.8333333333333334,0.005623


In [105]:
df.query('vg_img_id == 32')

,vg_img_id,cat,synset,vg_obj_name,responses_r0,opt-outs,vg_domain,top_response_domain_r0,responses_domains_r0,url,...,responses_r3,top_response_domain_r3,vg_object_id,all_responses,clean,canon,spellchecked,spellchecked_min2,verified,mn_name
21087,32,"instrumentality, instrumentation",furnishing.n.02,cabinet,"{'cabinet': 2, 'wood': 1, 'sink': 1, 'cabinets': 1, 'cabinetry': 1, 'modern kitchen': 1, 'wall': 1}",{'#bbox': 1},home,home,"{'home': 0.25, '$wood': 0.125, '$sink': 0.125, '$cabinets': 0.125, '$cabinetry': 0.125, '$modern kitchen': 0.125, '$wall': 0.125}",http://object-naming-amore.upf.edu//32_1058643_seed_ambiguous.png,...,"{'cabinet': 4, 'cabinets': 2, 'cupboards': 2, 'cupboard': 1}",home,1058643,"Counter({'cabinet': 15, 'cabinets': 9, 'cupboard': 3, 'wood': 2, 'cupboards': 2, 'sink': 1, 'cabinetry': 1, 'modern kitchen': 1, 'wall': 1})","{'cabinet': 15, 'cabinets': 9, 'cupboard': 3, 'wood': 2, 'cupboards': 2, 'sink': 1, 'cabinetry': 1, 'modern kitchen': 1, 'wall': 1}","{'cabinet': 15, 'cabinets': 9, 'cupboard': 5, 'wood': 2, 'sink': 1, 'cabinetry': 1, 'modern kitchen': 1, 'wall': 1}","Counter({'cabinet': 15, 'cabinets': 9, 'cupboard': 5, 'wood': 2, 'sink': 1, 'cabinetry': 1, 'modern kitchen': 1, 'wall': 1})","Counter({'cabinet': 15, 'cabinets': 9, 'cupboard': 5, 'wood': 2})","{'cabinets': {'cluster': ('cabinet', 'cabinets', 'cupboard'), 'can_be_same_object': ('cabinet', 'cabinets', 'cupboard'), 'adequacy': 1.0, 'inadequacy_type': None, 'cluster_id':...",cabinet


In [106]:
def get_error(topname,predname,verified,response):
    if predname == topname:
        return 'hit'
    if predname not in verified:
        if predname in response:
            return 'low_count'
        else:
            return 'off???'
    if topname in verified[predname]['can_be_same_object']:
        return 'adequate'
    if not verified[predname]['inadequacy_type']:
        return 'adequate_diff_object'
    
    return verified[predname]['inadequacy_type']

In [107]:
hum_error_cats = Counter()
sys_error_cats = Counter()
sys_error_cats2 = Counter()


for ix,row in dfsys.iterrows():
    
    rowh = df.query('vg_img_id == %d' % row['image_id']).iloc[0]
    resp = rowh['canon']
    topname = resp.most_common(1)[0][0]
    verif = rowh['verified']

    #print(topname)
    for name in resp:
        if name in vocab:
            err = get_error(topname,name,verif,resp)
            hum_error_cats[err] += resp[name]
        
    sys_err = get_error(topname,row['pred_name'],verif,resp)
    sys_error_cats[sys_err] += 1
    
    
for ix,row in dfsys2.iterrows():
    
    rowh = df.query('vg_img_id == %d' % row['image_id']).iloc[0]
    resp = rowh['canon']
    topname = resp.most_common(1)[0][0]
    verif = rowh['verified']
    sys_err = get_error(topname,row['pred_name'],verif,resp)
    sys_error_cats2[sys_err] += 1

In [121]:
relevant_cats = ['hit','adequate','low_count','bounding box','off???']
table = [
    ["humans"]+[hum_error_cats[t]/sum(hum_error_cats.values()) for t in relevant_cats],
    ["FRCNN, no finetuning"]+[sys_error_cats2[t]/sum(sys_error_cats2.values()) for t in relevant_cats],
    ["FRCNN, finetuning on MN"]+[sys_error_cats[t]/sum(sys_error_cats.values()) for t in relevant_cats]
]

In [122]:
pd.DataFrame(table,columns=['(442-vocab)']+relevant_cats)

,(442-vocab),hit,adequate,low_count,bounding box,off???
0,humans,0.758558,0.146164,0.042169,0.043843,0.000000
1,"FRCNN, no finetuning",0.749767,0.122316,0.023343,0.018674,0.077498
2,"FRCNN, finetuning on MN",0.788982,0.096172,0.017740,0.011204,0.079365


In [126]:
hum2_error_cats = Counter()
sys3_error_cats = Counter()
sys4_error_cats = Counter()


for ix,row in dfsys.iterrows():
    
    rowh = df.query('vg_img_id == %d' % row['image_id']).iloc[0]
    resp = rowh['canon']
    topname = rowh['vg_obj_name']
    verif = rowh['verified']

    #print(topname)
    for name in resp:
        if name in vocab:
            err = get_error(topname,name,verif,resp)
            hum2_error_cats[err] += resp[name]
        
    sys_err = get_error(topname,row['pred_name'],verif,resp)
    sys3_error_cats[sys_err] += 1
    
for ix,row in dfsys2.iterrows():
    
    rowh = df.query('vg_img_id == %d' % row['image_id']).iloc[0]
    resp = rowh['canon']
   
    topname = rowh['vg_obj_name']
    verif = rowh['verified']
    sys_err = get_error(topname,row['pred_name'],verif,resp)
    sys4_error_cats[sys_err] += 1

In [139]:
relevant_cats = ['hit','adequate','low_count','bounding box','off???']
table = [
   # ["VG","human",""]+[hum2_error_cats[t]/sum(hum2_error_cats.values()) for t in relevant_cats],
   # ["VG","FRCNN", "-"]+[sys4_error_cats[t]/sum(sys4_error_cats.values()) for t in relevant_cats],
   # ["VG","FRCNN", "+"]+[sys3_error_cats[t]/sum(sys3_error_cats.values()) for t in relevant_cats],
    ["MN","human",""]+[hum_error_cats[t]/sum(hum_error_cats.values()) for t in relevant_cats],
    ["MN","FRCNN", "-"]+[sys_error_cats2[t]/sum(sys_error_cats2.values()) for t in relevant_cats],
    ["MN","FRCNN", "+"]+[sys_error_cats[t]/sum(sys_error_cats.values()) for t in relevant_cats]
]
rdf = pd.DataFrame(table,columns=['Test on','model', 'finetuning']+relevant_cats)

In [140]:
rdf.round(3)

,Test on,model,finetuning,hit,adequate,low_count,bounding box,off???
0,MN,human,,0.759,0.146,0.042,0.044,0.000
1,MN,FRCNN,-,0.750,0.122,0.023,0.019,0.077
2,MN,FRCNN,+,0.789,0.096,0.018,0.011,0.079


In [ ]:
df.iloc½